In [2]:
import librosa
import panns_inference
from panns_inference import AudioTagging, SoundEventDetection, labels

/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
--2023-03-06 15:34:16--  http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/class_labels_indices.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 2607:f8b0:4004:c06::80, 2607:f8b0:4004:c09::80, 2607:f8b0:4004:c1b::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2607:f8b0:4004:c06::80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14675 (14K) [application/octet-stream]
Saving to: ‘/Users/andrewchang/panns_data/class_labels_indices.csv’

     0K .......... ....                                       100%  878K=0.02s

2023-03-06 15:34:16 (878 KB/s) - ‘/Users/andrewchang/panns_data/class_labels_indices.csv’ saved [14675/14675]



In [4]:
import numpy as np
import pandas as pd
import random as python_random
df_all = pd.read_csv('../train_val_test_split/train_val_test_GoogleAudioSet.csv', index_col=0)
df_all

,file,source,category,weight,fold
0,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,0
1,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,8
2,../data/interim/GoogleAudioSet_unbalanced_list...,Google_nature,0,1,5
3,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,1
4,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,1
...,...,...,...,...,...
13662,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,5
13663,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,3
13664,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,8
13665,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,8


In [23]:
import pickle
    
file = open(df_all.loc[0,'file'], 'rb')
output = pickle.load(file)
wav_raw = output['y']
wav_bg = output['bg_y']
wav_fg = output['fg_y']

In [28]:
wav_raw.shape

(149778,)

In [27]:
wav_all = np.stack([wav_raw, wav_bg, wav_fg], axis=0)
wav_all.shape

(3, 149778)

In [25]:
at = AudioTagging(checkpoint_path=None, device='cuda')

Checkpoint path: /Users/andrewchang/panns_data/Cnn14_mAP=0.431.pth
Using CPU.


In [10]:
# wav_raw = wav_raw[None, :]
# wav_raw.shape

(1, 149778)

In [30]:
import time

In [35]:
start_time = time.time()
(clipwise_output, embedding) = at.inference(wav_raw[None, :])
print(time.time()-start_time)

0.17624807357788086


In [36]:
start_time = time.time()
(clipwise_output, embedding) = at.inference(wav_all)
print(time.time()-start_time)

0.2648899555206299


In [38]:
clipwise_output.shape

(3, 527)

In [13]:
embedding.shape

(1, 2048)

In [21]:
len(at.labels)

527